# 2.3 WaveFunction Overlap - Randomized Measurement V1

The `WaveFunctionOverlap` or `EchoListen` instance provides three methods:

1. **`EchoListenRandomized`**: Uses randomized measurements and serves as the default method.
   ```python
   exp_method_02 = WaveFunctionOverlap()
   ```
   or
   ```python
   exp_method_02 = EchoListen()
   ```
   or
   ```python
   exp_method_02 = WaveFunctionOverlap(method="randomized")
   ```
   or
   ```python
   exp_method_02 = EchoListen(method="randomized")
   ```
   or
   ```python
   exp_method_02 = WaveFunctionOverlap(method="base")
   ```
   or
   ```python
   exp_method_02 = EchoListen(method="base")
   ```
2. **`EchoListenHadamard`**: Employs the Hadamard test for entropy calculation.
   ```python
   exp_method_01 = WaveFunctionOverlap(method="hadamard")
   ```
   or
   ```python
   exp_method_01 = EchoListen(method="hadamard")
   ```
3. **`EchoListenRandomizedV1`**: Implements randomized measurements with a different approach but is deprecated due to limitations in subsystem selection.
   ```python
   exp_method_03 = WaveFunctionOverlap(method="randomized_v1")
   ```
   or
   ```python
   exp_method_03 = EchoListen(method="randomized_v1")
   ```

All three methods calculate the entropy of a quantum state, but their calculation techniques and post-processing procedures are entirely different.

## 2.3 Randomized Measurement V1

This method is based on [Probing Rényi entanglement entropy via randomized measurements](https://www.science.org/doi/abs/10.1126/science.aau4963) with deplorizing error mitigation by [Simple mitigation of global depolarizing errors in quantum simulations](https://link.aps.org/doi/10.1103/PhysRevE.104.035309).


---

## `randomized_overlap_echo_v1`

<a id='randomized_overlap_echo_v1'></a>


Here, we assume you already have the data from the calculation. You can call the function `randomized_overlap_echo_v1` from `qurry.process.randomized_measure` to calculate the entropy.

In this function, only `shots`, `counts`, and `degree` are required arguments. The others are optional.

The following is the arguments of the function:

```python
def randomized_overlap_echo_v1(
    shots: int,
    counts: list[dict[str, int]],
    degree: Optional[Union[tuple[int, int], int]] = None,
    measure: Optional[tuple[int, int]] = None,
    backend: PostProcessingBackendLabel = DEFAULT_PROCESS_BACKEND,
    workers_num: Optional[int] = None,
    pbar: Optional[tqdm.tqdm] = None,
) -> dict[str, float]:
    """Calculate wavefunction overlap
    a.k.a. loschmidt echo when processes time evolution system.

    Args:
        shots (int):
            Shots of the counts.
        counts (list[dict[str, int]]):
            Counts from randomized measurement results.
        degree (Optional[Union[tuple[int, int], int]]):
            The range of partition.
        measure (Optional[tuple[int, int]], optional):
            The range that implemented the measuring gate.
            If not specified, then use all qubits.
            This will affect the range of partition
            when you not implement the measuring gate on all qubit.
            Defaults to None.
        backend (PostProcessingBackendLabel, optional):
            Backend for the post-processing.
            Defaults to DEFAULT_PROCESS_BACKEND.
        workers_num (Optional[int], optional):
            Number of multi-processing workers, it will be ignored if backend is Rust.
            if sets to 1, then disable to using multi-processing;
            if not specified, then use the number of all cpu counts by `os.cpu_count()`.
            This only works for Python and Cython backend.
            Defaults to None.
        pbar (Optional[tqdm.tqdm], optional):
            The progress bar API, you can use put a :cls:`tqdm` object here.
            This function will update the progress bar description.
            Defaults to None.

    Returns:
        dict[str, float]: A dictionary contains purity, entropy,
            a list of each overlap, puritySD, degree, actual measure range, bitstring range.
    """
```


---

### Dummy Data


In [1]:
from qurry.capsule import quickRead

easy_dummy: dict[str, dict[str, int]] = quickRead("../easy-dummy.json")
large_dummy_list = [easy_dummy["0"] for _ in range(100)]

---

### Simple Example


In [2]:
from qurry.process.randomized_measure import randomized_overlap_echo_v1

test_result_1_1_1 = randomized_overlap_echo_v1(
    4096, [*large_dummy_list, *large_dummy_list], 6
)

In [3]:
from pprint import pprint

print("| result of randomized_overlap_echo_v1 except for purityCells")
pprint({k: v for k, v in test_result_1_1_1.items() if k != "echoCells"})
# "echoCells" is too long we skip it here

print("| result of randomized_overlap_echo_v1[purityCells]")
print(test_result_1_1_1["echoCells"][0])
print(test_result_1_1_1["echoCells"][1])

| result of randomized_overlap_echo_v1 except for purityCells
{'bitStringRange': (2, 8),
 'countsNum': 200,
 'degree': 6,
 'echo': np.float64(1.0627930164337158),
 'echoSD': np.float64(0.0),
 'measureActually': (2, 8),
 'takingTime': 0.006928145}
| result of randomized_overlap_echo_v1[purityCells]
1.0627930164337158
1.0627930164337158


---

### Integration with your own progress bar


In [4]:
from tqdm import tqdm
import time

all_counts_progress_01 = tqdm(
    [
        (4096, large_dummy_list, 6),
        (4096, large_dummy_list, (2, 8)),
        (4096, large_dummy_list, 7),
        (4096, large_dummy_list, (0, 7)),
        (4096, large_dummy_list, (-2, 5)),
        (4096, large_dummy_list, (-5, -1)),
        (4096, large_dummy_list, (3, -2)),
    ],
    bar_format="| {desc} - {elapsed} < {remaining}",
)

test_result_1_1_2 = []
for tmp_shot, tmp_counts, tmp_partition in all_counts_progress_01:
    test_result_1_1_2.append(
        randomized_overlap_echo_v1(
            tmp_shot,
            [*tmp_counts, *tmp_counts],
            tmp_partition,
            pbar=all_counts_progress_01,
        )
    )
    time.sleep(0.5)

| Calculate overlap with 200 counts. - 00:03 < 00:00


---

### Using Python backend

It will be slow. You'd better think twice before using it.


In [5]:
all_counts_progress_02 = tqdm(
    [
        (4096, large_dummy_list, 6),
        (4096, large_dummy_list, (2, 8)),
        (4096, large_dummy_list, 7),
        (4096, large_dummy_list, (0, 7)),
        (4096, large_dummy_list, (-2, 5)),
        (4096, large_dummy_list, (-5, -1)),
        (4096, large_dummy_list, (3, -2)),
    ],
    bar_format="| {desc} - {elapsed} < {remaining}",
)


test_result_1_1_2 = []
for tmp_shot, tmp_counts, tmp_partition in all_counts_progress_02:
    test_result_1_1_2.append(
        randomized_overlap_echo_v1(
            tmp_shot,
            [*tmp_counts, *tmp_counts],
            tmp_partition,
            pbar=all_counts_progress_02,
            backend="Python",
        )
    )
    time.sleep(0.5)

| Calculate overlap with 200 counts. - 00:12 < 00:00


---

## Post-Processing Speed-up Availablities and Version Info



In [6]:
from qurry.process.status import AVAIBILITY_STATESHEET

AVAIBILITY_STATESHEET

 | Qurry version: 0.10.0
--------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust  
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes   
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes   
   - wavefunction_overlap.echo_core_2 ....... Yes    Depr.  Yes   
   - wavefunction_overlap.echo_cell_2 ....... Yes    Depr.  Error 
 - utils
   - randomized ............................. Yes    Depr.  Yes   
   - construct .............................. Yes    No     Yes   
   - dummy .................................. Yes    No     Yes   
   - test ................................... Yes    No     Yes   
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes   
 - magnet_square
   - magnsq_core ............................ Yes    No     No    
--------------------------------------------------------
   + Yes ...... Working normally